In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import the necessary libraries
import time
import os
log_dir = os.path.join(os.curdir, 'logs')

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from skimage import io, util
from skimage.color import rgba2rgb
from tensorflow import keras

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

#define other properties
font_size = 25
font_color = 'white'
plt.rc('axes', facecolor = '#1e1e1e')
plt.rc('figure', facecolor = '#1e1e1e')
plt.rc('axes', edgecolor = 'w')
plt.rc('xtick', color = 'w')
plt.rc('ytick', color = 'w')
plt.rc('legend', facecolor = 'w')

In [ ]:
# define the image path
import pathlib
#img_train_folder = pathlib.Path('/content/drive/MyDrive/dataset/pictures/train/128x128') 
img_train_folder = pathlib.Path('/content/drive/MyDrive/dataset/pictures/train/64x64') 

In [ ]:
# auxiliary code to visualize the first 10 pictures of each category
plt.figure(figsize = (50,50))
j = 0
counter1 = 0
counter2 = 0

for dir1 in os.listdir(img_train_folder):
    for file in os.listdir(os.path.join(img_train_folder, dir1)):
        image_path = os.path.join(img_train_folder, dir1, file)  
        img = mpimg.imread(image_path)
        ax = plt.subplot(10, 10, j+1)
        ax.title.set_text(file)
        plt.axis('off')
        plt.title(dir1, fontsize = font_size, color = font_color)
        plt.imshow(img)
        j += 1
        counter1 += 1
        if counter1 == 10:
            break
    counter2 += 1
    counter1 = 0
    if counter2 == 7:
        break

In [ ]:
# function to gather pictures and generate a numpy array (GRAYSCALE)
def create_dataset(img_folder):
   
    counter = 0
    img_data_array = []
    class_name = []
   
    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):
            image_path = os.path.join(img_folder, dir1, file)
            image = io.imread(image_path, as_gray = True)
            image = util.invert(image)
            image = np.array(image, dtype = 'float32')
            image /= 255
            img_data_array.append(image)
            class_name.append(int(dir1))
            counter += 1
    return img_data_array, class_name, counter

In [ ]:
# function to gather pictures and generate a numpy array (COLOR)
def create_dataset(img_folder):
   
    counter = 0
    img_data_array = []
    class_name = []
   
    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):
            image_path = os.path.join(img_folder, dir1, file)
            image = io.imread(image_path, as_gray = False)
            image = np.array(image)
            image = rgba2rgb(image)                      
            img_data_array.append(image)
            class_name.append(int(dir1))
            counter += 1
    return img_data_array, class_name, counter

In [ ]:
train_test_size = 0.75
test_valid_size = 0.5
random_state = 42

# dataset import
X_array1, y_array1, counter = create_dataset(img_train_folder)

X_trainSeq = np.array(X_array1)
y_trainSeq = np.array(y_array1)

X_train, X_test, y_train, y_test = train_test_split(X_trainSeq, y_trainSeq, train_size = train_test_size, random_state = random_state, stratify = y_trainSeq)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, train_size = test_valid_size, random_state = random_state, stratify = y_test)

print('treinamento completo:', X_train.shape)
print('validação:           ', X_valid.shape)
print('testes:              ', X_test.shape)

In [ ]:
## CNN grayscale only ##
X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

print('treinamento completo:', X_train.shape)
print('validação:           ', X_valid.shape)
print('testes:              ', X_test.shape)

In [ ]:
# checks if all categories are evenly distributed
unique, counts = np.unique(y_train, return_counts = True)
print(dict(zip(unique, counts)))

unique, counts = np.unique(y_valid, return_counts = True)
print(dict(zip(unique, counts)))

unique, counts = np.unique(y_test, return_counts = True)
print(dict(zip(unique, counts)))

In [ ]:
# enumerate the classes found
class_names = ['cpu', 'gpu', 'mobo', 'ram', 'hd', 'ssd-sata', 'ssd-m2']

In [ ]:
# train dataset visualization (100 first items)
n_rows = 5
n_cols = 20
plt.figure(figsize = (n_cols * 1.2, n_rows * 1.2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(X_train[index, :, :], cmap = 'binary')
        plt.axis('off')
        plt.title(class_names[y_train[index]], fontsize = font_size / 2, color = font_color)
plt.subplots_adjust(wspace = 0.2, hspace = 0.5)
plt.show()

In [ ]:
# validation dataset visualization (100 first items)
n_rows = 5
n_cols = 20
plt.figure(figsize = (n_cols * 1.2, n_rows * 1.2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(X_valid[index, :, :], cmap = 'binary')
        plt.axis('off')
        plt.title(class_names[y_valid[index]], fontsize = font_size / 2, color = font_color)
plt.subplots_adjust(wspace = 0.2, hspace = 0.5)
plt.show()

In [ ]:
# test dataset visualization (100 first items)
n_rows = 5
n_cols = 20
plt.figure(figsize = (n_cols * 1.2, n_rows * 1.2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(X_test[index, :, :], cmap = 'binary')
        plt.axis('off')
        plt.title(class_names[y_test[index]], fontsize = font_size / 2, color = font_color)
plt.subplots_adjust(wspace = 0.2, hspace = 0.5)
plt.show()

In [ ]:
# visualization of the first instance from the train dataset
plt.imshow(X_train[0], cmap = 'binary')
plt.show()

In [ ]:
# visualization of the first instance from the validation dataset
plt.imshow(X_valid[0], cmap = 'binary')
plt.show()

In [ ]:
# visualization of the first instance from the test dataset
plt.imshow(X_test[0], cmap = 'binary')
plt.show()

In [ ]:
# auxiliary code to visualize the array generated from the pictures
#np.set_printoptions(threshold=sys.maxsize)
print(X_train[0])
#print(X_valid[0])
#print(X_test[0])

In [ ]:
# auxiliary code to visualize the label of the picture
#np.set_printoptions(threshold=sys.maxsize)
print(y_train[0])
#print(y_valid[0])
#print(y_test[0])

In [ ]:
# shows the class of the first instance of the train dataset
class_names[y_train[0]]

In [ ]:
# shows the class of the first instance of the validation dataset
class_names[y_valid[0]]

In [ ]:
# shows the class of the first instance of the test dataset
class_names[y_test[0]]

In [ ]:
## DNN ##
dimensionR     = 64
dimensionC     = 64
classes        = 7
neurons        = 10
activation     = 'relu'
activation_out = 'softmax'
learning_rate  = 0.3
channels       = 1
epochs         = 500
callbacks      = []     # [keras.callbacks.EarlyStopping(patience=10)]

# build the neural network layers
keras.backend.clear_session() # clears the session
np.random.seed(random_state) # random numpy seed, but fixed
tf.random.set_seed(random_state) # random tf seed, but fixed

model = keras.models.Sequential([
            keras.layers.Flatten(input_shape = [dimensionR, dimensionC, channels]),
            keras.layers.Dense(neurons * 100, activation = activation),
            keras.layers.Dense(neurons * 10, activation = activation),
            keras.layers.Dense(neurons, activation = activation),            
            keras.layers.Dense(classes, activation = activation_out)
        ])
model.summary()

In [ ]:
## CNN ##
dimensionR     = 64
dimensionC     = 64
classes        = 7
filters        = 8
activation     = 'relu'
activation_out = 'softmax'
kernel_size    = 3
strides        = 1
padding        = 'same'
channels       = 3
epochs         = 100
callbacks      = []     # [keras.callbacks.EarlyStopping(patience=10)]

# build the neural network layers
keras.backend.clear_session() # clears the session
np.random.seed(random_state) # random numpy seed, but fixed
tf.random.set_seed(random_state) # random tf seed, but fixed

model = keras.models.Sequential([
                keras.layers.Conv2D(filters, kernel_size = kernel_size, strides = strides, padding = padding, activation = activation, input_shape = [dimensionR, dimensionC, channels]),
                keras.layers.MaxPool2D(pool_size = 2),
                keras.layers.Conv2D(filters * 2, kernel_size = kernel_size, strides = strides, padding = padding, activation = activation),
                keras.layers.Flatten(),
                keras.layers.Dropout(0.25),
                keras.layers.Dense(filters * 4, activation = activation),
                keras.layers.Dropout(0.5),
                keras.layers.Dense(7, activation = activation_out)
        ])
model.summary()

In [ ]:
# prints the DNN/CNN structure
tf.keras.utils.plot_model(model, 'pc_hardware_model.png', show_shapes = True)

In [ ]:
# gets weights and biases of the layer 1
hidden1 = model.layers[1]
weights, biases = hidden1.get_weights()
print('weights:', weights.shape)
print('biases: ', biases.shape)

In [ ]:
# shows all  the weights of the DNN
weights

In [ ]:
# shows all  the biases of the DNN
biases

In [ ]:
# sets the learning rate of the optimizer (DNN)
sgd = tf.keras.optimizers.SGD(learning_rate = learning_rate)
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])

run_dir1 = os.path.join(log_dir, 'sgd' + str(learning_rate) + '-' + 'eph' + str(epochs) + '-' + 'img' + str(counter) + '-' + str(dimensionR) + 'x' + str(dimensionC) + '-' + time.strftime('run_%Y_%m_%d-%H_%M_%S'))
tensorboard1 = keras.callbacks.TensorBoard(run_dir1)

# trains the neural network
%time history = model.fit(X_train, y_train, epochs = epochs, validation_data = (X_valid, y_valid), callbacks = [tensorboard1])

In [ ]:
# sets NADAM as the optimizer (CNN)
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'nadam', metrics = ['accuracy'])

run_dir1 = os.path.join(log_dir, 'nadam' + '-' + 'eph' + str(epochs) + '-' + 'img' + str(counter) + '-' + str(dimensionR) + 'x' + str(dimensionC) + '-' + time.strftime('run_%Y_%m_%d-%H_%M_%S'))
tensorboard1 = keras.callbacks.TensorBoard(run_dir1)

# trains the neural network
%time history = model.fit(X_train, y_train, epochs = epochs, validation_data = (X_valid, y_valid), callbacks = [tensorboard1])

In [ ]:
# prints the convergence graph
pd.DataFrame(history.history).plot(figsize=(12, 6))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

In [ ]:
# prints the error data
print('erro de treino:   ', history.history['loss'][-1])
print('erro de validação:', history.history['val_loss'][-1])
print('erro de teste:    ', model.evaluate(X_test, y_test, verbose = 0))

In [ ]:
# loads Tensorboard
%load_ext tensorboard
%tensorboard --logdir=./logs --port=6006

In [ ]:
# probabilities for the first 200 test instances
X_new = X_test[:200]
y_proba = model.predict(X_new)
print(y_proba.round(2))

In [ ]:
# predicted classes for the same 200 test instances
y_pred = np.argmax(model.predict(X_new), axis = -1)
print('previstas: ', np.array(class_names)[y_pred])
print('reais:     ', np.array(class_names)[y_test[:200]])

In [ ]:
# test dataset evaluation view
n_rows = 5
n_cols = 20
plt.figure(figsize = (n_cols * 1.2, n_rows * 1.2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(X_new[index, :, :], cmap = 'binary')
        plt.axis('off')
        plt.title(class_names[y_pred[index]], fontsize = font_size / 2, color = font_color)
plt.subplots_adjust(wspace = 0.2, hspace = 0.5)
plt.show()

In [ ]:
# saves the DNN neural network
model.save('sgd' + str(learning_rate) + '-' + 'eph' + str(epochs) + '-' + 'img' + str(counter) + '-' + str(dimensionR) + 'x' + str(dimensionC) + '-' + time.strftime('run_%Y_%m_%d-%H_%M_%S') + '.h5')

In [ ]:
# saves the CNN neural network
model.save('nadam' + '-' + 'eph' + str(epochs) + '-' + 'img' + str(counter) + '-' + str(dimensionR) + 'x' + str(dimensionC) + '-' + time.strftime('run_%Y_%m_%d-%H_%M_%S') + '.h5')